In [1]:
import os,sys,inspect
from ipywidgets import interact, interactive, fixed, interact_manual, Button
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
from datetime import date
import ipywidgets as widgets
import ipython_blocking
import numpy as np
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )

testname="Проверка устойчивости к механическому воздействию"
display(Markdown('### 💠 {}'.format(testname)))

### 💠 Проверка устойчивости к механическому воздействию

In [2]:
from connect_device_v2.find_device import FindDevice
box = FindDevice()
display(box)

Выбранная конфигурация
filter_if:10 kHz
point_value:200
athenuator_value:10
cap_value:6554


<IPython.core.display.Javascript object>

In [11]:
button = box.select_device_continue_button

In [12]:
%blockrun button

In [13]:
button1 = box.select_config_continue_button

In [14]:
%blockrun button1

In [15]:
inst = box.selected_device

In [16]:
inst.write("SYST:PRES")

10

In [17]:
result_config =  box.config

In [18]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun 21 09:28:15 2021

@author: kuzma.m
"""

import pyvisa as visa

#функция декоратор
def command_wrapper(f):
    def wrapped(inst, command):
        # print('До функции')
        try:
            if key == "debug":
                print(command)
        except:
            pass
        response = f(inst, command)
        # print('После функции')
        error = inst.query('SYST:ERR?')
        if error != '0, No error':
            print(error)
        return response

    # print('декорируем', f)
    return wrapped

#запрос данных
@command_wrapper
def query(inst, command):
    result = inst.query(command)
    return result

@command_wrapper
def query_ascii_values(inst, command):
    result = inst.query_ascii_values(command)
    return result

#запрос данных
@command_wrapper
def write(inst, command):
    inst.write(command)

#Функция конвертации множителей kGz,MGh,GHz в герцы
def HzConvertor(n,k):
    if k == "kHz" or k == "кГц":
        return int(n*1e+3)
    elif k == "MHz" or k == "МГц":
        return int(n*1e+6)
    elif k == "GHz" or k == "ГГц":
        return int(n*1e+9)
    else: return -1

#Функция однократного росчерка
def singlescan(inst):
    write(inst, f'TRIG:SOUR BUS')
    write(inst, f'INIT:CONT OFF')
    write(inst, f'INIT')
    write(inst, f'TRIG:SING')
    query(inst, f'*OPC?')

In [19]:
value_dict = {}

In [20]:
def tablesort():
    global value_dict
    temp_dict = {}
    for i in range(1,int(query(inst,f'SERV:PORT:COUN?'))+1):
        try:
            temp_dict[f'S{i}{i}'] = value_dict[f'S{i}{i}'] 
        except KeyError:
            pass    
    value_dict = {}
    value_dict = temp_dict

In [21]:
out = widgets.HTML()


In [22]:
def parse_config(config: dict):
    new_config = config.copy()
    
    if_split = config['DEVICE']['filter_if'].split(" ")
    if_value = f'{HzConvertor(int(if_split[0]),if_split[1])}'
    
    
    at_value = config['DEVICE']['athenuator_value']
    poin_value = config['DEVICE']['point_value']
    cap_value = config['DEVICE']['cap_value']
    
    
    new_config['DEVICE']['if_value']=if_value
    new_config['DEVICE']['athenuator_value']=at_value
    new_config['DEVICE']['point_value'] = poin_value
    new_config['DEVICE']['cap_value'] = cap_value
    return new_config
    

In [23]:
new_config=parse_config(dict(result_config))

In [24]:
def param(new_config):
    #Пресет
    write(inst, f'SYST:PRES')
    
    #Устанавливаем линейное сканирование
    write(inst, f'SENS:SWE:TYPE LIN')
    
    #Запрашиваем полный диапазон частот прибора и устанавливаем
    #freq_max = query(CMT, f'SERV:SWE:FREQ:MAX?')
    #freq_min = query(CMT, f'SERV:SWE:FREQ:MAX?')
    
    
    #Attenuator control – вкл. 
    write(inst, f'SERV:RFCTL:POW:STAT 1')
    write(inst, f'SERV:RFCTL:POW:ATT {new_config["DEVICE"]["athenuator_value"]}')

    #Установить фильтр ПЧ
    write(inst, f'SENS:BWID {new_config["DEVICE"]["if_value"]}')
    time.sleep(1) 

    #установить Код ЦАП.
    write(inst, f'SERV:RFCTL:POW:DAC {new_config["DEVICE"]["cap_value"]}')
    
    #установить кол-во точек.
    write(inst, f'SENS:SWE:POIN {new_config["DEVICE"]["point_value"]}')


In [25]:
def setport(port):
    
    write(inst, f'CALC1:PAR:COUN 1')
    write(inst, f'CALC1:PAR1:DEF S{port}{port}')

In [26]:
def normalize():
    write(inst,f'CALC:PAR:SEL')
    write(inst,f'CALC:MATH:MEM')
    write(inst,f'CALC:MATH:FUNC DIV')
    

In [27]:
def gridbutton(count):
    row = 0
    port_count = 0
    
    if count%6 == 0:
        row_1 = count//6
    else:
        row_1 = count//6+1
    c = 6
    grid = widgets.GridspecLayout(row_1 * 3, 6)
    for i in range(row_1):
        if i == row_1 - 1:
            c = count - c * i
        for j in range(c):
            port_count += 1
            g1 = widgets.Button(description=f'До воздействия')
            g1.on_click(functools.partial(before_on_button_clicked, rs_=f'{port_count}'))

            g2 =  widgets.Button(description=f'После воздействия')
            g2.on_click(functools.partial(after_on_button_clicked, rs_=f'{port_count}'))
            grid[row + 0, j] = widgets.Label(value=f'port {port_count}')
            grid[row + 1, j] = g1
            grid[row + 2, j] = g2
        row += 3
    display(grid)


In [28]:
#Устанавливаем параметры и выставляем порты
param(new_config)

date_inf = f'{date.today()}'
time_inf = query(inst,f'SYST:TIME?')

In [29]:
def marksearch():
        
    trace = query(inst, f'CALC1:PAR1:DEF?')
    write(inst, f'CALC1:TRAC1:MARK1:STAT ON')
    write(inst, f'CALC1:TRAC1:MARK2:STAT ON')
    write(inst, f'CALC:MARK1:FUNC:TYPE MAX')
    write(inst, f'CALC:MARK2:FUNC:TYPE MIN')

    write(inst, f'CALC:MARK1:FUNC:EXEC')
    write(inst, f'CALC:MARK2:FUNC:EXEC')

    max_value = abs(query_ascii_values(inst, f'CALC:MARK1:Y?')[0])
    min_value = abs(query_ascii_values(inst, f'CALC:MARK2:Y?')[0])
    if max_value > min_value:
        value_dict.update({f'{trace}': max_value})
    else:
        value_dict.update({f'{trace}': min_value})

In [30]:
from IPython.display import HTML, display

def tablehtml(): 
    #создаем строку для ввода в html страницу
    htmlstr = "<table border='1' ><tr><td>Порт</td><td>Отклонение</td><td>Максимум, дБ</td></tr><tr>"
    count = 0
    MAX = -1
    for trace in value_dict.keys():
        if value_dict[f'{trace}'] > MAX:
            MAX = value_dict[f'{trace}']
    for trace in value_dict.keys():
        trace_value = value_dict[f'{trace}']
        cut = (len(trace)-1)//2 + 1
        port_data = trace[cut:]
        if trace_value == MAX:
            htmlstr += f'<td>{port_data}</td><td class = bg-warning>{trace_value}</td>'
        else:
            htmlstr += f'<td>{port_data}</td><td>{trace_value}</td>'
        if count == 0:
            htmlstr += f'<td rowspan={len(value_dict.keys())+1}>{MAX}</td>'
            count += 1
        htmlstr += "</tr><tr>"
    
    return htmlstr



In [31]:
def teblehader():
    
    idn_inf = query(inst,f'*IDN?').split(', ')
    #idn_inf = "".join(idn_inf).split(", ")
    htmlstr = "<table border='1' ><tr><td> Дата запуска </td><td> Время запуска </td><td>  device </td><td> model </td><td> serial </td><td>  version </td></tr><tr>"
    htmlstr += f'<td>{date_inf}</td><td>{time_inf}</td><td>{idn_inf[0]}</td><td>{idn_inf[1]}</td><td>{idn_inf[2]}</td><td>{idn_inf[3]}</td>'
    return htmlstr

In [32]:
def before_on_button_clicked(b, rs_="some_default_string"):
    setport(rs_)
    singlescan(inst)
    normalize()
    b.button_style = 'success'

In [33]:
def after_on_button_clicked(b, rs_="some_default_string"):
    setport(rs_)
    singlescan(inst)
    marksearch()
    out.value = teblehader() + tablehtml()
    b.button_style = 'success'
    global save_button
    save_button.file_data = teblehader() + tablehtml()


In [34]:
def sort_on_button_clicked(b):
    tablesort()
    out.value = teblehader() + tablehtml()
    global save_button
    save_button.file_data = teblehader() + tablehtml()

In [35]:

gridbutton(int(query(inst,f'SERV:PORT:COUN?')))
display(out)
save_button = SaveFilesButton(out.value)
sort_button =  widgets.Button(description=f'Сортировать таблицу')
sort_button.on_click(sort_on_button_clicked)
display(sort_button)
display(save_button)


GridspecLayout(children=(Label(value='port 1', layout=Layout(grid_area='widget001')), Button(description='До в…

HTML(value='')

Button(description='Сортировать таблицу', style=ButtonStyle())

SaveFilesButton(description='SaveFiles', icon='square-o', style=ButtonStyle(button_color='orange'))

In [36]:
#cont_button = widgets.Button(description=f'Сохранить')
#display(cont_button)

In [37]:
#%blockrun cont_button